In [1]:
from datetime import datetime
import logging
from multiprocessing import Pool

import numpy as np
import pandas as pd

import os
from PIL import Image


In [146]:
from loader import dataloader
from pathmgmt import pathmgmt as myPath
from alpha_perfmc import metrics
from alpha_perfmc import utility
from logger import logger as myLogger

In [147]:
import warnings
warnings.filterwarnings('ignore')


In [148]:
logger = myLogger.Logger(__name__)
logger.init(console_handler=True)

In [149]:
class PnLCalculator():
    def __init__(self, args_dict):
        self.__name = args_dict['table_name']
        self.__universe = self.__name.split('-')[-1]
        self.__start = args_dict['start_date']
        self.__end = args_dict['end_date']
        self.__alpha = self.__load_alpha()
        self.__transform = args_dict['transform']
        self.__neutralize = args_dict['neutralize']
        self.__holding = args_dict['holding']
        self.__longname = f'{self.__name};{self.__start}-{self.__end}-{self.__transform}-{self.__neutralize}-holding{self.__holding}days'
        
    def __load_alpha(self):
        logger().debug("loading computed alphas...")
        alphas = dataloader.loading(
            tab_name=self.__name, start=self.__start, end=self.__end, fields=['code', 'name', 'alpha', 'fut_ret_1d'])
        logger().debug("loading computed alphas...Done!")
        return alphas
    
    def compute_pnl(self):
        # convert alphas into positions
        self.__alpha = utility.alpha_to_weight(self.__alpha, self.__transform, self.__neutralize)
        # compute rolling alphas for holding period larger than 1
        self.__alpha = utility.rolling(self.__alpha, self.__holding)
        # compute pnl

In [150]:
def save(df, dates, tableName):
    tableDir = myPath.ALPHA_DIR/tableName
    tableDir.mkdir(parents=True, exist_ok=True)
    logger().debug(f"Start writing to files for alpha {tableName}...")

    for date in dates:
        outputfile = myPath.ALPHA_DIR/tableName/(date+'.csv')
        if outputfile.exists():
            return
        try:
            df1d = df.loc[df.index == date].reset_index()
            df1d.time = df1d.time.dt.strftime("%Y%m%d")
            df1d = df1d[['time', 'code', 'name', 'alpha', 'fut_ret_1d']]
            df1d.to_csv(outputfile, index=False)
            # logger().info(f"Writing to file on date {date} for alpha {tab}...Done!")
        except:
            logger().error(
                f"Exception when Writing to file on date {date} for alpha {tableName}")

    logger().debug("Writing to files...Done!")

In [170]:
def read_alphas(config):
    res = pd.DataFrame()
    for config in args_dict_list:
        calcobj = PnLCalculator(config)
        calcobj.compute_pnl()
        alphas = calcobj._PnLCalculator__alpha
        alphas = alphas[['code', 'name', 'weights', 'fut_ret_1d']]
        alphas.rename(columns={'weights': calcobj._PnLCalculator__longname}, inplace=True)
        if res.empty:
            res = alphas
        else:
            res = res.reset_index().merge(alphas.reset_index(), on=['time', 'code', 'name', 'fut_ret_1d'], how='inner').set_index('time')
    return res

In [223]:
def combine_alphas(res, category='PV', method='equalweight', weights_map=None):
    alpha_cols = [c for c in res.columns if c not in ['code', 'name', 'fut_ret_1d'] and 'alpha' in c]
    res[method] = 0
    for col in alpha_cols:
        res[method] += res[col] * weights_map[col] / sum(weights_map.values())
    dates = [d.strftime("%Y%m%d") for d in res.index.unique()]
    dates.sort()
    # frist date set to NaN
    res.loc[res.index == dates[0], method] = np.nan
    combined_alpha = res[['code', 'name', method, 'fut_ret_1d']].copy(deep=True)
    combined_alpha.rename(columns={method: 'alpha'}, inplace=True)
    save(combined_alpha, dates, f"combined-{category}-{method}-zz9999")

In [154]:
def combine_images(columns, space, images, name):
    rows = len(images) // columns
    if len(images) % columns:
        rows += 1
    width_max = max([Image.open(image).width for image in images])
    height_max = max([Image.open(image).height for image in images])
    background_width = width_max*columns + (space*columns)-space
    background_height = height_max*rows + (space*rows)-space
    background = Image.new('RGBA', (background_width, background_height), (255, 255, 255, 255))
    x = 0
    y = 0
    for i, image in enumerate(images):
        img = Image.open(image)
        x_offset = int((width_max-img.width)/2)
        y_offset = int((height_max-img.height)/2)
        background.paste(img, (x+x_offset, y+y_offset))
        x += width_max + space
        if (i+1) % columns == 0:
            y += height_max + space
            x = 0
    background.save(name)

In [155]:
def plot_combined(category, method):
    alphaName = f'alpha.combined-{category}-{method}-zz9999'
    files = []
    for folderName in os.listdir(myPath.PLOT_DIR):
        if alphaName in folderName:
            for subfolder in os.listdir(myPath.PLOT_DIR/folderName):
                for file in os.listdir(myPath.PLOT_DIR/folderName/subfolder):
                    files.append(myPath.PLOT_DIR/folderName/subfolder/file)
    (myPath.PLOT_DIR/alphaName/'summary').mkdir(parents=True, exist_ok=True)
    for holdings in [1, 3, 5, 10, 20, 60]:
        images = [f for f in files if f'None-Industry-holding{holdings}days' in str(f) and 'png' in str(f)]
        combine_images(columns=1, space=20, images=images, name=myPath.PLOT_DIR/alphaName/'summary'/f'summary.{alphaName};None-Industry-holding{holdings}days.png')

### Basic PV: Momentum and Reversion

In [59]:
args_dict_list = []
args_dict_list.append({
    'table_name': f'alpha.YaoMoM004-60days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoMoM007-5days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoReV001-60days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoReV005-20days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoReV009-10days-a3-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'Rank',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoReV009-3days-a5-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'Rank',
    'neutralize': 'Industry',
    'holding': 1
})

In [ ]:
### Equally weighted
equalweight = {'alpha.YaoMoM004-60days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
             'alpha.YaoMoM007-5days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
             'alpha.YaoReV001-60days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
             'alpha.YaoReV005-20days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
             'alpha.YaoReV009-10days-a3-zz9999;20180101-20201231-Rank-Industry-holding1days': 1,
             'alpha.YaoReV009-3days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days': 1}

In [88]:
### IR weighted
irweights = {'alpha.YaoMoM004-60days-zz9999;20180101-20201231-None-Industry-holding1days': 5.103285,
             'alpha.YaoMoM007-5days-zz9999;20180101-20201231-None-Industry-holding1days': 6.552738,
             'alpha.YaoReV001-60days-zz9999;20180101-20201231-None-Industry-holding1days': 2.451816,
             'alpha.YaoReV005-20days-zz9999;20180101-20201231-None-Industry-holding1days': 2.898292,
             'alpha.YaoReV009-10days-a3-zz9999;20180101-20201231-Rank-Industry-holding1days': 3.202771,
             'alpha.YaoReV009-3days-a5-zz9999;20180101-20201231-Rank-Industry-holding1days': 3.471707}

In [ ]:
### IC weighted?

In [ ]:
alphas = read_alphas(args_dict_list)

In [ ]:
combine_alphas(alphas, 'PV', 'equalweight', equalweight)

In [ ]:
combine_alphas(alphas, 'PV', 'irweight', irweights)

In [156]:
plot_combined('PV', 'equalweight')

In [157]:
plot_combined('PV', 'irweight')

### Technical

In [158]:
args_dict_list = []
args_dict_list.append({
    'table_name': f'alpha.YaoTec003-3days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoTec006-5days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoTec007-3days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoTec007-60days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoTec008-20days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoTec011-10days-10days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'Rank',
    'neutralize': 'Industry',
    'holding': 1
})


In [ ]:
alphas_tec = read_alphas(args_dict_list)

In [172]:
### Equally weighted
equalweight = {'alpha.YaoTec003-3days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoTec006-5days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoTec007-3days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoTec007-60days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoTec008-20days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoTec011-10days-10days-zz9999;20180101-20201231-Rank-Industry-holding1days': 1}


In [173]:
### IR weighted
irweights = {'alpha.YaoTec003-3days-zz9999;20180101-20201231-None-Industry-holding1days': 4.506186,
             'alpha.YaoTec006-5days-zz9999;20180101-20201231-None-Industry-holding1days': 2.274438,
             'alpha.YaoTec007-3days-zz9999;20180101-20201231-None-Industry-holding1days':  2.885066,
             'alpha.YaoTec007-60days-zz9999;20180101-20201231-None-Industry-holding1days': 3.349317,
             'alpha.YaoTec008-20days-zz9999;20180101-20201231-None-Industry-holding1days': 2.521566,
             'alpha.YaoTec011-10days-10days-zz9999;20180101-20201231-Rank-Industry-holding1days': 3.027162}


In [175]:
combine_alphas(alphas_tec, 'Tec', 'equalweight', equalweight)

2023-06-21 22:20:14,706:DEBUG:__main__:Start writing to files for alpha combined-Tec-equalweight-zz9999...
2023-06-21 22:20:14,706:DEBUG:__main__:Start writing to files for alpha combined-Tec-equalweight-zz9999...
2023-06-21 22:20:44,531:DEBUG:__main__:Writing to files...Done!
2023-06-21 22:20:44,531:DEBUG:__main__:Writing to files...Done!


In [176]:
combine_alphas(alphas_tec, 'Tec', 'irweight', irweights)

2023-06-21 22:22:18,886:DEBUG:__main__:Start writing to files for alpha combined-Tec-irweight-zz9999...
2023-06-21 22:22:18,886:DEBUG:__main__:Start writing to files for alpha combined-Tec-irweight-zz9999...
2023-06-21 22:22:52,338:DEBUG:__main__:Writing to files...Done!
2023-06-21 22:22:52,338:DEBUG:__main__:Writing to files...Done!


In [177]:
plot_combined('Tec', 'equalweight')

In [178]:
plot_combined('Tec', 'irweight')

### Volatility

In [179]:
args_dict_list = []
args_dict_list.append({
    'table_name': f'alpha.YaoVol002-20days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoVol005-10days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoVol006-60days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoVol007-20days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'Rank',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoVol011-20days-60days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'Rank',
    'neutralize': 'Industry',
    'holding': 1
})


In [ ]:
alphas_vol = read_alphas(args_dict_list)

In [182]:
### Equally weighted
equalweight = {'alpha.YaoVol002-20days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoVol005-10days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoVol006-60days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoVol007-20days-zz9999;20180101-20201231-Rank-Industry-holding1days': 1,
               'alpha.YaoVol011-20days-60days-zz9999;20180101-20201231-Rank-Industry-holding1days': 1}


In [183]:
irweights = {'alpha.YaoVol002-20days-zz9999;20180101-20201231-None-Industry-holding1days': 2.956636,
             'alpha.YaoVol005-10days-zz9999;20180101-20201231-None-Industry-holding1days':  2.803632,
             'alpha.YaoVol006-60days-zz9999;20180101-20201231-None-Industry-holding1days':   3.184895,
             'alpha.YaoVol007-20days-zz9999;20180101-20201231-Rank-Industry-holding1days': 2.810485,
             'alpha.YaoVol011-20days-60days-zz9999;20180101-20201231-Rank-Industry-holding1days': 2.243600}

In [184]:
combine_alphas(alphas_vol, 'Vol', 'equalweight', equalweight)

2023-06-21 22:57:16,619:DEBUG:__main__:Start writing to files for alpha combined-Vol-equalweight-zz9999...
2023-06-21 22:57:16,619:DEBUG:__main__:Start writing to files for alpha combined-Vol-equalweight-zz9999...
2023-06-21 22:57:43,696:DEBUG:__main__:Writing to files...Done!
2023-06-21 22:57:43,696:DEBUG:__main__:Writing to files...Done!


In [185]:
combine_alphas(alphas_vol, 'Vol', 'irweight', irweights)

2023-06-21 22:58:19,985:DEBUG:__main__:Start writing to files for alpha combined-Vol-irweight-zz9999...
2023-06-21 22:58:19,985:DEBUG:__main__:Start writing to files for alpha combined-Vol-irweight-zz9999...
2023-06-21 22:58:46,776:DEBUG:__main__:Writing to files...Done!
2023-06-21 22:58:46,776:DEBUG:__main__:Writing to files...Done!


In [186]:
plot_combined('Vol', 'equalweight')

In [187]:
plot_combined('Vol', 'irweight')

### Liquidity

In [195]:
args_dict_list = []
args_dict_list.append({
    'table_name': f'alpha.YaoLqd001-10days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'Rank',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoLqd006-20days-a05-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoLqd009-3days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoLqd015-3days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})

In [ ]:
alphas_lqd = read_alphas(args_dict_list)

In [197]:
### Equally weighted
equalweight = {'alpha.YaoLqd001-10days-zz9999;20180101-20201231-Rank-Industry-holding1days': 1,
               'alpha.YaoLqd006-20days-a05-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoLqd009-3days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoLqd015-3days-zz9999;20180101-20201231-None-Industry-holding1days': 1}

In [198]:
### IR weighted
irweights = {'alpha.YaoLqd001-10days-zz9999;20180101-20201231-Rank-Industry-holding1days': 4.674443,
             'alpha.YaoLqd006-20days-a05-zz9999;20180101-20201231-None-Industry-holding1days': 2.283723,
             'alpha.YaoLqd009-3days-zz9999;20180101-20201231-None-Industry-holding1days': 5.505706,
               'alpha.YaoLqd015-3days-zz9999;20180101-20201231-None-Industry-holding1days': 4.291100}

In [199]:
combine_alphas(alphas_lqd, 'Lqd', 'equalweight', equalweight)

2023-06-21 23:34:35,791:DEBUG:__main__:Start writing to files for alpha combined-Lqd-equalweight-zz9999...
2023-06-21 23:34:35,791:DEBUG:__main__:Start writing to files for alpha combined-Lqd-equalweight-zz9999...
2023-06-21 23:35:09,561:DEBUG:__main__:Writing to files...Done!
2023-06-21 23:35:09,561:DEBUG:__main__:Writing to files...Done!


In [200]:
combine_alphas(alphas_lqd, 'Lqd', 'irweight', irweights)

2023-06-21 23:35:14,505:DEBUG:__main__:Start writing to files for alpha combined-Lqd-irweight-zz9999...
2023-06-21 23:35:14,505:DEBUG:__main__:Start writing to files for alpha combined-Lqd-irweight-zz9999...
2023-06-21 23:35:48,213:DEBUG:__main__:Writing to files...Done!
2023-06-21 23:35:48,213:DEBUG:__main__:Writing to files...Done!


In [201]:
plot_combined('Lqd', 'equalweight')

In [202]:
plot_combined('Lqd', 'irweight')

### Money Flow

In [203]:
args_dict_list = []
args_dict_list.append({
    'table_name': f'alpha.YaoMfl002-20days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoMfl003-5days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoMfl005-60days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoMfl008-3days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})
args_dict_list.append({
    'table_name': f'alpha.YaoMfl016-60days-zz9999',
    'start_date': '20180101',
    'end_date': '20201231',
    'transform': 'None',
    'neutralize': 'Industry',
    'holding': 1
})

In [ ]:
alphas_mfl = read_alphas(args_dict_list)

In [205]:
### Equally weighted
equalweight = {'alpha.YaoMfl002-20days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoMfl003-5days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoMfl008-3days-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.YaoMfl016-60days-zz9999;20180101-20201231-None-Industry-holding1days': 1}

In [206]:
### IR weighted
irweights = {'alpha.YaoMfl002-20days-zz9999;20180101-20201231-None-Industry-holding1days': 2.612018,
             'alpha.YaoMfl003-5days-zz9999;20180101-20201231-None-Industry-holding1days': 2.007744,
             'alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding1days': 3.713988,
             'alpha.YaoMfl008-3days-zz9999;20180101-20201231-None-Industry-holding1days': 5.589981,
               'alpha.YaoMfl016-60days-zz9999;20180101-20201231-None-Industry-holding1days': 3.238267}

In [207]:
combine_alphas(alphas_mfl, 'Mfl', 'equalweight', equalweight)

2023-06-22 00:04:44,722:DEBUG:__main__:Start writing to files for alpha combined-Mfl-equalweight-zz9999...
2023-06-22 00:04:44,722:DEBUG:__main__:Start writing to files for alpha combined-Mfl-equalweight-zz9999...
2023-06-22 00:05:13,629:DEBUG:__main__:Writing to files...Done!
2023-06-22 00:05:13,629:DEBUG:__main__:Writing to files...Done!


In [208]:
combine_alphas(alphas_mfl, 'Mfl', 'irweight', irweights)

2023-06-22 00:06:22,411:DEBUG:__main__:Start writing to files for alpha combined-Mfl-irweight-zz9999...
2023-06-22 00:06:22,411:DEBUG:__main__:Start writing to files for alpha combined-Mfl-irweight-zz9999...
2023-06-22 00:06:52,055:DEBUG:__main__:Writing to files...Done!
2023-06-22 00:06:52,055:DEBUG:__main__:Writing to files...Done!


In [209]:
plot_combined('Mfl', 'equalweight')


In [210]:
plot_combined('Mfl', 'irweight')

### Level 2

In [211]:
args_dict_list = []
for cat in ['Lqd', 'Tec', 'Vol', 'PV', 'Mfl']:
    args_dict_list.append({
        'table_name': f'alpha.combined-{cat}-equalweight-zz9999',
        'start_date': '20180101',
        'end_date': '20201231',
        'transform': 'None',
        'neutralize': 'Industry',
        'holding': 1
    })
    args_dict_list.append({
        'table_name': f'alpha.combined-{cat}-irweight-zz9999',
        'start_date': '20180101',
        'end_date': '20201231',
        'transform': 'None',
        'neutralize': 'Industry',
        'holding': 1
    })

In [ ]:
alphas_lvl2 = read_alphas(args_dict_list)

In [213]:
alphas_lvl2

,code,name,alpha.combined-Lqd-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,fut_ret_1d,alpha.combined-Lqd-irweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Tec-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Tec-irweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Vol-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Vol-irweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-PV-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-PV-irweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Mfl-equalweight-zz9999;20180101-20201231-None-Industry-holding1days,alpha.combined-Mfl-irweight-zz9999;20180101-20201231-None-Industry-holding1days
time,,,,,,,,,,,,,
2018-09-21,1,平安银行,-0.001590,-0.011310,-0.001411,-0.000130,-0.000130,0.000000e+00,0.000000e+00,-0.000219,0.000051,-0.001258,-0.001293
2018-09-21,2,万科A,-0.004653,-0.046003,-0.004694,0.000099,0.000100,0.000000e+00,0.000000e+00,0.000595,0.000694,-0.002434,-0.002428
2018-09-21,4,国华网安,-0.001354,-0.006851,-0.001317,-0.000605,-0.000546,0.000000e+00,0.000000e+00,-0.000261,-0.000187,-0.000163,-0.000247
2018-09-21,5,ST星源,-0.000916,-0.023065,-0.000920,0.000204,0.000292,0.000000e+00,0.000000e+00,-0.000836,-0.000665,-0.000326,-0.000374
2018-09-21,6,深振业A,-0.001083,-0.054997,-0.001252,-0.002230,-0.002127,0.000000e+00,0.000000e+00,-0.000132,0.000284,-0.000828,-0.001017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31,688333,铂力特,-0.000035,NaN,-0.000001,-0.000345,-0.000514,8.113234e-18,9.217583e-18,0.000657,0.000914,-0.000413,-0.000595
2020-12-31,688366,昊海生科,-0.000416,NaN,-0.000465,-0.000247,-0.000183,1.149014e-17,7.086636e-18,0.000042,0.000002,-0.000016,-0.000271
2020-12-31,688368,晶丰明源,0.000093,NaN,0.000057,-0.000080,-0.000275,1.167119e-17,4.337204e-18,-0.000403,-0.000436,-0.000031,-0.000354


In [224]:
### Equally weighted
equalweight = {'alpha.combined-Lqd-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-Lqd-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-Mfl-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-Mfl-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-PV-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-PV-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-Tec-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-Tec-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-Vol-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 1,
               'alpha.combined-Vol-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 1}

In [225]:
### IR weighted
irweights = {'alpha.combined-Lqd-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 7.040704,
               'alpha.combined-Lqd-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 7.122243,
               'alpha.combined-Mfl-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 5.456058,
               'alpha.combined-Mfl-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 5.769453,
               'alpha.combined-PV-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 10.655995,
               'alpha.combined-PV-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 9.719494,
               'alpha.combined-Tec-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 5.453688,
               'alpha.combined-Tec-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 5.425224,
               'alpha.combined-Vol-equalweight-zz9999;20180101-20201231-None-Industry-holding1days': 4.707861,
               'alpha.combined-Vol-irweight-zz9999;20180101-20201231-None-Industry-holding1days': 4.848018}

In [226]:
combine_alphas(alphas_lvl2, 'lvl2', 'equalweight', equalweight)

2023-06-22 01:00:22,204:DEBUG:__main__:Start writing to files for alpha combined-lvl2-equalweight-zz9999...
2023-06-22 01:00:22,204:DEBUG:__main__:Start writing to files for alpha combined-lvl2-equalweight-zz9999...
2023-06-22 01:00:46,240:DEBUG:__main__:Writing to files...Done!
2023-06-22 01:00:46,240:DEBUG:__main__:Writing to files...Done!


In [227]:
combine_alphas(alphas_lvl2, 'lvl2', 'irweight', irweights)

2023-06-22 01:00:51,593:DEBUG:__main__:Start writing to files for alpha combined-lvl2-irweight-zz9999...
2023-06-22 01:00:51,593:DEBUG:__main__:Start writing to files for alpha combined-lvl2-irweight-zz9999...
2023-06-22 01:01:15,195:DEBUG:__main__:Writing to files...Done!
2023-06-22 01:01:15,195:DEBUG:__main__:Writing to files...Done!


In [228]:
plot_combined('lvl2', 'equalweight')

In [229]:
plot_combined('lvl2', 'irweight')